# VISUALIZANDO CLUSTERS CON UNA ALTA DIMENSIONALIDAD

## Introducción

En este notebook, exploraremos dos métodos diferentes que se pueden utilizar para visualizar los clústeres que se formaron en datos de alta dimensionalidad (datos con más de tres dimensiones).

Visualizaremos nuestros cuatro clústeres utilizando nuestros dos métodos: Análisis de Componentes Principales (PCA) y T-Distributed Stochastic Neighbor Embedding (T-SNE).

## Importamos las librerias

In [1]:
#Basic imports
import numpy as np
import pandas as pd

#sklearn imports
from sklearn.decomposition import PCA #Principal Component Analysis
from sklearn.manifold import TSNE #T-Distributed Stochastic Neighbor Embedding
from sklearn.cluster import KMeans #K-Means Clustering
from sklearn.preprocessing import StandardScaler #used for 'Feature Scaling'

#plotly imports
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

#load model an pipeline
from joblib import load

## Leemos nuestros datos

In [2]:
df = pd.read_csv('../../data_clean.csv').drop(columns=['Unnamed: 0'])
df

,price,Año de fabricación,Kilometraje,Transmisión,Combustible,Cilindrada,Categoría,Marca,Tracción,Número cilindros,link,cluster
0,25500.0,2018.0,25800.0,Automática - Secuencial,Gasolina,1500.0,Camionetas Suv,Peugeot,4x2,4.0,https://neoauto.com/auto/usado/peugeot-5008-20...,3
1,16900.0,2012.0,29000.0,Automática - Secuencial,Gasolina,2400.0,Camionetas Suv,Jeep,AWD,4.0,https://neoauto.com/auto/usado/jeep-compass-20...,1
2,22500.0,2017.0,53000.0,Automática - Secuencial,Gasolina,1500.0,Sedan,Bmw,Posterior,4.0,https://neoauto.com/auto/usado/bmw-318i-2017-1...,3
3,10500.0,2020.0,35000.0,Mecánica,Gasolina,1400.0,Camionetas Suv,Changhe,4x2,4.0,https://neoauto.com/auto/usado/changhe-m50s-20...,3
4,20200.0,2019.0,55954.0,Automática - Secuencial,Gasolina,1500.0,Camionetas Suv,Toyota,4x2,4.0,https://neoauto.com/auto/usado/toyota-rush-201...,3
...,...,...,...,...,...,...,...,...,...,...,...,...
5272,20000.0,2014.0,86000.0,Mecánica,Dual,2400.0,Camionetas Suv,Hyundai,4x2,4.0,https://neoauto.com/auto/usado/hyundai-santa-f...,1
5273,10500.0,2018.0,85000.0,Automática,Dual,1368.0,Sedan,Kia,4x2,4.0,https://neoauto.com/auto/usado/kia-new-rio-201...,3
5274,14900.0,2017.0,58999.0,Automática - Secuencial,Dual,2000.0,Sedan,Mazda,Delantera,4.0,https://neoauto.com/auto/usado/mazda-3-2017-16...,1
5275,23000.0,2017.0,60000.0,Mecánica,Gasolina,2000.0,Camionetas Suv,Toyota,4x2,4.0,https://neoauto.com/auto/usado/toyota-rav4-201...,1


In [3]:
# Me quedo con solo las varibles numericas que usé para el entrenamiento
numeric_cols = ['price', 'Año de fabricación', 'Kilometraje', 'Cilindrada']
df_num = df.loc[:, numeric_cols]
scaler = StandardScaler()
df_num[numeric_cols] = scaler.fit_transform(X=df_num)
df_num['cluster'] = df['cluster'].values
df_num

,price,Año de fabricación,Kilometraje,Cilindrada,cluster
0,1.000976,0.366616,-0.880186,-1.037077,3
1,-0.153255,-1.257534,-0.788099,1.336087,1
2,0.598337,0.095925,-0.097441,-1.037077,3
3,-1.012217,0.908000,-0.615434,-1.300761,3
4,0.289648,0.637308,-0.012433,-1.037077,3
...,...,...,...,...,...
5272,0.262805,-0.716151,0.852213,1.336087,1
5273,-1.012217,0.366616,0.823436,-1.385141,3
5274,-0.421680,0.095925,0.075195,0.281348,1
5275,0.665444,0.095925,0.104001,0.281348,1


## **Metodo \#1:** Análisis de componentes principales

Para nuestra primera visualización usaremos Principal Component Analysis (PCA).

PCA es un algoritmo que se utiliza para la reducción de dimensionalidad, lo que significa, de manera informal, que puede tomar un DataFrame con muchas columnas y devolver un DataFrame con un número reducido de columnas que aún retiene gran parte de la información de las columnas del DataFrame original. Las columnas del DataFrame producido por el procedimiento de PCA se llaman Componentes Principales. Utilizaremos estos componentes principales para ayudarnos a visualizar nuestros clústeres en espacio 1-D, 2-D y 3-D, ya que no podemos visualizar fácilmente los datos que tenemos en dimensiones mayores. Por ejemplo, podemos utilizar dos componentes principales para visualizar los clústeres en el espacio 2-D, o tres componentes principales para visualizar los clústeres en el espacio 3-D.

Pero primero vamos a crear una copia de nuestro dataframe donde vamos a trabajar

In [4]:
plotX = df.copy()

Ahora para visualizar nuestros datos vamos a crear 3 dataframes usando el algoritmo de PCA.

El primer DataFrame contendrá los resultados del algoritmo PCA con solo un componente principal. Este DataFrame se utilizará para visualizar nuestros clústeres en una dimensión (1-D).

El segundo DataFrame contendrá los dos componentes principales devueltos por el algoritmo PCA con n_components=2. Este DataFrame nos ayudará en nuestra visualización de estos clústeres en dos dimensiones (2-D).

Y el tercer DataFrame contendrá los resultados del algoritmo PCA que devuelve tres componentes principales. Este DataFrame nos permitirá visualizar los clústeres en el espacio tridimensional (3-D).

In [5]:
#PCA con un componente principal
pca_1d = PCA(n_components=1)

#PCA con dos componentes principales
pca_2d = PCA(n_components=2)

#PCA con tres componentes principales
pca_3d = PCA(n_components=3)

Cronstruimos nuestros nuevos dataframes:

In [6]:
#Este DataFrame tendrá ese solo componente principal como se mecionó antes
PCs_1d = pd.DataFrame(pca_1d.fit_transform(df_num.drop(["cluster"], axis=1)))

#Este DataFrame contiene los dos componentes principales que serán usados
#por la visualization 2-D
PCs_2d = pd.DataFrame(pca_2d.fit_transform(df_num.drop(["cluster"], axis=1)))

#y este DataFrame contiene tres componentes princiopales que usaremos en
#nuestra visualización de nuestros clusters in 3-D
PCs_3d = pd.DataFrame(pca_3d.fit_transform(df_num.drop(["cluster"], axis=1)))

Renombremos las columnas de estos nuevos dataframes que hemos creado

In [7]:
PCs_1d.columns = ["PC1_1d"]

PCs_2d.columns = ["PC1_2d", "PC2_2d"]

PCs_3d.columns = ["PC1_3d", "PC2_3d", "PC3_3d"]

Concatenamos estos nuevos dataframes con el dataframe que usaremos para realizar las visualizaciones

In [8]:
plotX = pd.concat([plotX, PCs_1d, PCs_2d, PCs_3d], axis=1, join='inner')
plotX

,price,Año de fabricación,Kilometraje,Transmisión,Combustible,Cilindrada,Categoría,Marca,Tracción,Número cilindros,link,cluster,PC1_1d,PC1_2d,PC2_2d,PC1_3d,PC2_3d,PC3_3d
0,25500.0,2018.0,25800.0,Automática - Secuencial,Gasolina,1500.0,Camionetas Suv,Peugeot,4x2,4.0,https://neoauto.com/auto/usado/peugeot-5008-20...,3,-1.274582,-1.274582,-0.625942,-1.274582,-0.625942,0.517831
1,16900.0,2012.0,29000.0,Automática - Secuencial,Gasolina,2400.0,Camionetas Suv,Jeep,AWD,4.0,https://neoauto.com/auto/usado/jeep-compass-20...,1,0.418133,0.418133,1.072102,0.418133,1.072102,-1.446211
2,22500.0,2017.0,53000.0,Automática - Secuencial,Gasolina,1500.0,Sedan,Bmw,Posterior,4.0,https://neoauto.com/auto/usado/bmw-318i-2017-1...,3,-0.436705,-0.436705,-0.608104,-0.436705,-0.608104,0.767949
3,10500.0,2020.0,35000.0,Mecánica,Gasolina,1400.0,Camionetas Suv,Changhe,4x2,4.0,https://neoauto.com/auto/usado/changhe-m50s-20...,3,-0.445777,-0.445777,-1.854343,-0.445777,-1.854343,-0.214395
4,20200.0,2019.0,55954.0,Automática - Secuencial,Gasolina,1500.0,Camionetas Suv,Toyota,4x2,4.0,https://neoauto.com/auto/usado/toyota-rush-201...,3,-0.569572,-0.569572,-0.828448,-0.569572,-0.828448,0.744600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5272,20000.0,2014.0,86000.0,Mecánica,Dual,2400.0,Camionetas Suv,Hyundai,4x2,4.0,https://neoauto.com/auto/usado/hyundai-santa-f...,1,0.844754,0.844754,1.539430,0.844754,1.539430,-0.029936
5273,10500.0,2018.0,85000.0,Automática,Dual,1368.0,Sedan,Kia,4x2,4.0,https://neoauto.com/auto/usado/kia-new-rio-201...,3,0.747399,0.747399,-1.525079,0.747399,-1.525079,0.690212
5274,14900.0,2017.0,58999.0,Automática - Secuencial,Dual,2000.0,Sedan,Mazda,Delantera,4.0,https://neoauto.com/auto/usado/mazda-3-2017-16...,1,0.201832,0.201832,0.029551,0.201832,0.029551,-0.299819
5275,23000.0,2017.0,60000.0,Mecánica,Gasolina,2000.0,Camionetas Suv,Toyota,4x2,4.0,https://neoauto.com/auto/usado/toyota-rav4-201...,1,-0.328998,-0.328998,0.563682,-0.328998,0.563682,0.326024


Crearemos una nueva columna para nuestro dataframe que lo usaremos en la visualización 1-D

In [9]:
plotX['dummy'] = 0

Ahora dividiremos nuestro dataframe, `df`, en 4 nuevos dataframes.

Cada uno de estos nuevos DataFrames contendrá todos los valores contenidos en exactamente uno de los clústeres. Por ejemplo, todos los valores contenidos dentro del DataFrame, cluster0 pertenecerán al 'clúster 0', y todos los valores contenidos en el DataFrame, cluster1 pertenecerán al 'clúster 1', etc.

In [10]:
# Es importante mencionar que todos los DataFrames mencionados anteriormente son sub-DataFrames de 'plotX'.
# Esto se debe a que nuestra intención es graficar los valores contenidos dentro de cada uno de estos DataFrames.

cluster0 = plotX[plotX["cluster"] == 0]
cluster1 = plotX[plotX["cluster"] == 1]
cluster2 = plotX[plotX["cluster"] == 2]
cluster3 = plotX[plotX["cluster"] == 3]

## Visualización con PCA:

In [11]:
#Esto es necesario para que podamos mostrar los graficos de plotly apropiadamente
init_notebook_mode(connected=True)

### Visualización 1-D

El grafico de abajo muestra nuestros cuatro clusters originales en una sola visualización 1-D de sus componentes principales

In [12]:
#Instrucciones para construir el grafico 1-D

#trace1 es para 'Cluster 0'
trace1 = go.Scatter(
                    x = cluster0["PC1_1d"],
                    y = cluster0["dummy"],
                    mode = "markers",
                    name = "Cluster 0",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text = None)

#trace2 es para 'Cluster 1'
trace2 = go.Scatter(
                    x = cluster1["PC1_1d"],
                    y = cluster1["dummy"],
                    mode = "markers",
                    name = "Cluster 1",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text = None)

#trace3 es para 'Cluster 2'
trace3 = go.Scatter(
                    x = cluster2["PC1_1d"],
                    y = cluster2["dummy"],
                    mode = "markers",
                    name = "Cluster 2",
                    marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
                    text = None)

#trace3 es para 'Cluster 3'
trace4 = go.Scatter(
                    x = cluster3["PC1_1d"],
                    y = cluster3["dummy"],
                    mode = "markers",
                    name = "Cluster 3",
                    marker = dict(color = 'rgba(20, 80, 160, 0.8)'),
                    text = None)

data = [trace1, trace2, trace3, trace4]

title = "Visualizando los Clusters en una Sola Dimension Usando PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= '',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

### Visualización 2-D

In [13]:
#Instrucciones para construir el gráfico 2-D

#trace1 es para 'Cluster 0'
trace1 = go.Scatter(
                    x = cluster0["PC1_2d"],
                    y = cluster0["PC2_2d"],
                    mode = "markers",
                    name = "Cluster 0",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text = None)

#trace2 es para 'Cluster 1'
trace2 = go.Scatter(
                    x = cluster1["PC1_2d"],
                    y = cluster1["PC2_2d"],
                    mode = "markers",
                    name = "Cluster 1",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text = None)

#trace3 es para 'Cluster 2'
trace3 = go.Scatter(
                    x = cluster2["PC1_2d"],
                    y = cluster2["PC2_2d"],
                    mode = "markers",
                    name = "Cluster 2",
                    marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
                    text = None)

#trace4 es para 'Cluster 3'
trace4 = go.Scatter(
                    x = cluster3["PC1_2d"],
                    y = cluster3["PC2_2d"],
                    mode = "markers",
                    name = "Cluster 3",
                    marker = dict(color = 'rgba(20, 80, 160, 0.8)'),
                    text = None)

data = [trace1, trace2, trace3, trace4]

title = "Visualizando los Clusters en Dos Dimensiones Usando PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

### Visualización 3-D

In [14]:
#Intrucciones para construir el grafico 3-D

#trace1 es para 'Cluster 0'
trace1 = go.Scatter3d(
                    x = cluster0["PC1_3d"],
                    y = cluster0["PC2_3d"],
                    z = cluster0["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 0",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text = None)

#trace2 es para 'Cluster 1'
trace2 = go.Scatter3d(
                    x = cluster1["PC1_3d"],
                    y = cluster1["PC2_3d"],
                    z = cluster1["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 1",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text = None)

#trace3 es para 'Cluster 2'
trace3 = go.Scatter3d(
                    x = cluster2["PC1_3d"],
                    y = cluster2["PC2_3d"],
                    z = cluster2["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 2",
                    marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
                    text = None)

#trace4 es para 'Cluster 3'
trace4 = go.Scatter3d(
                    x = cluster3["PC1_3d"],
                    y = cluster3["PC2_3d"],
                    z = cluster3["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 3",
                    marker = dict(color = 'rgba(20, 80, 160, 0.8)'),
                    text = None)

data = [trace1, trace2, trace3, trace4]

title = "Visualizando los Clusters en Tres Dimensiones Usando PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

### Comentarios sobre PCA:

Como podemos ver en los gráficos anteriores: si se tiene datos altamente agrupables, PCA es una buena forma de ver los clústeres formados en los datos originales. Además, parece que la visualización de los clústeres es más efectiva cuando los clústeres se visualizan utilizando más componentes principales, en lugar de menos. Por ejemplo, el gráfico 2-D hizo un mejor trabajo proporcionando una representación visual clara de los clústeres que el gráfico 1-D; y el gráfico 3-D hizo un mejor trabajo que el gráfico 2-D.

## **Method \#2:** T-Distributed Stochastic Neighbor Embedding (T-SNE):

Nuestro próximo método para visualizar nuestros clústeres es [T-Distributed Stochastic Neighbor Embedding (T-SNE)](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding).

Aquí hay un buen [video](https://www.youtube.com/watch?v=wvsE8jm1GzE) de Google que da una rápida descripción de lo que hace el algoritmo. Y aquí hay un [video](https://www.youtube.com/watch?v=NEaUSP4YerM) que da una explicación útil y simplificada de cómo el algoritmo hace lo que hace, si estas interesado.

En resumen, T-SNE es un algoritmo de aprendizaje automático interesante y complejo que nos puede ayudar a visualizar datos de alta dimensionalidad. Es un método para realizar reducción de dimensionalidad, y es por esta razón que podemos usarlo para ayudarnos a visualizar nuestros tres clústeres que se construyeron en datos de alta dimensionalidad.


Nota: Y al igual que antes, utilizaremos este algoritmo para visualizar nuestros datos en espacio 1-D, 2-D y 3-D.

Una vez más crearemos una copia de nuestro dataframe llamada `plotX` para poder visualizar nuestros clusters.

In [15]:
plotX = df.copy()

A continuación, tenemos que decidir qué nivel de `perplejidad` deseamos utilizar para nuestro algoritmo T-SNE. La `perplejidad` es un hiperparámetro utilizado en el algoritmo T-SNE que determina en gran medida cómo se distribuyen los datos devueltos por el algoritmo.

Para ver el papel que juega la `perplejidad` en la forma en que se distribuyen los datos a través de T-SNE, consulte este [artículo](https://distill.pub/2016/misread-tsne/) claramente escrito e interactivo de algunos de los ingenieros/científicos de [Google Brain](https://ai.google/research/teams/brain).

He encontrado, a través de unas pocas pruebas, que `perplejidad` = 50 funciona bastante bien para estos datos, pero estoy convencido de que probablemente existe un valor más ideal para la `perplejidad` entre los valores de 30 y 50. Si estás dispuesto a aceptar el desafío, no dudes en editar este notebook e intenta encontrar el valor de `perplejidad` que mejor muestre los clústeres formados en los datos originales.

In [16]:
#Fijamos nuestra perplejidad
perplexity = 50

Inicializamos nuestros modelos de T-SNE:

In [17]:
#T-SNE con una dimensión
tsne_1d = TSNE(n_components=1, perplexity=perplexity)

#T-SNE con dos dimensión
tsne_2d = TSNE(n_components=2, perplexity=perplexity)

#T-SNE con tres dimensión
tsne_3d = TSNE(n_components=3, perplexity=perplexity)

Construimos nuestros nuevos dataframes paara ayudarnos a visualizar nuestra data en el espacio 1-D, 2-D y 3-D:

In [18]:
#Este Dataframe tendrá una sola dimension, construido por T-SNE
TCs_1d = pd.DataFrame(tsne_1d.fit_transform(df_num.drop(["cluster"], axis=1)))

#Este Dataframe contendrá two dimensiones, construido por T-SNE
TCs_2d = pd.DataFrame(tsne_2d.fit_transform(df_num.drop(["cluster"], axis=1)))

#Y este DataFrame contendrá tres dimensiones, construido por T-SNE
TCs_3d = pd.DataFrame(tsne_3d.fit_transform(df_num.drop(["cluster"], axis=1)))

Renombramos las columnas de estos nuevos Dataframes creados:

In [19]:
TCs_1d.columns = ["TC1_1d"]

#"TC1_2d" significa: 'El primer componente de las componentes creadas para la visualización 2-D, por T-SNE.'
#And "TC2_2d" significa: 'El segundo componente de las componentes creadas para la visualización 2-D, por T-SNE.'
TCs_2d.columns = ["TC1_2d","TC2_2d"]

TCs_3d.columns = ["TC1_3d","TC2_3d","TC3_3d"]

Concatenaremos estos nuevos Dataframes a `plotX` para poder usarlos como columnas propias de este.

In [20]:
plotX = pd.concat([plotX,TCs_1d,TCs_2d,TCs_3d], axis=1, join='inner')

Y crearemos una nueva columnas en `plotX` para poder usarla en nuestra visualización 1-D

In [21]:
plotX["dummy"] = 0

Ahora dividiremos nuestro Dataframe, `plotX`, en 4 nuevos Dataframes

In [22]:
cluster0 = plotX[plotX["cluster"] == 0]
cluster1 = plotX[plotX["cluster"] == 1]
cluster2 = plotX[plotX["cluster"] == 2]
cluster3 = plotX[plotX["cluster"] == 3]

## Visualizaciones T-SNE:

### Visualización 1-D:

In [23]:
#Instrucciones para construir la visualización 1-D

#trace1 es para 'Cluster 0'
trace1 = go.Scatter(
                    x = cluster0["TC1_1d"],
                    y = cluster0["dummy"],
                    mode = "markers",
                    name = "Cluster 0",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text = None)

#trace2 es para 'Cluster 1'
trace2 = go.Scatter(
                    x = cluster1["TC1_1d"],
                    y = cluster1["dummy"],
                    mode = "markers",
                    name = "Cluster 1",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text = None)

#trace3 es para 'Cluster 2'
trace3 = go.Scatter(
                    x = cluster2["TC1_1d"],
                    y = cluster2["dummy"],
                    mode = "markers",
                    name = "Cluster 2",
                    marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
                    text = None)

#trace4 es para 'Cluster 3'
trace4 = go.Scatter(
                    x = cluster3["TC1_1d"],
                    y = cluster3["dummy"],
                    mode = "markers",
                    name = "Cluster 3",
                    marker = dict(color = 'rgba(20, 80, 160, 0.8)'),
                    text = None)

data = [trace1, trace2, trace3, trace4]

title = "Visualisando Clusters en Una Dimension Usando T-SNE (perplexity=" + str(perplexity) + ")"

layout = dict(title = title,
              xaxis= dict(title= 'TC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= '',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

### Visualización 2-D:

In [24]:
#Instrucciones para construir la visualización 2-D

#trace1 es para 'Cluster 0'
trace1 = go.Scatter(
                    x = cluster0["TC1_2d"],
                    y = cluster0["TC2_2d"],
                    mode = "markers",
                    name = "Cluster 0",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text = None)

#trace2 es para 'Cluster 1'
trace2 = go.Scatter(
                    x = cluster1["TC1_2d"],
                    y = cluster1["TC2_2d"],
                    mode = "markers",
                    name = "Cluster 1",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text = None)

#trace3 es para 'Cluster 2'
trace3 = go.Scatter(
                    x = cluster2["TC1_2d"],
                    y = cluster2["TC2_2d"],
                    mode = "markers",
                    name = "Cluster 2",
                    marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
                    text = None)

#trace4 es para 'Cluster 3'
trace4 = go.Scatter(
                    x = cluster3["TC1_2d"],
                    y = cluster3["TC2_2d"],
                    mode = "markers",
                    name = "Cluster 3",
                    marker = dict(color = 'rgba(20, 80, 160, 0.8)'),
                    text = None)

data = [trace1, trace2, trace3, trace4]

title = "Visualisando Clusters en Dos Dimensiones Usando T-SNE (perplexity=" + str(perplexity) + ")"

layout = dict(title = title,
              xaxis= dict(title= 'TC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'TC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

### Visualización 3-D:

In [25]:
#Instrucciones para construir la visualización 3-D

#trace1 es para 'Cluster 0'
trace1 = go.Scatter3d(
                    x = cluster0["TC1_3d"],
                    y = cluster0["TC2_3d"],
                    z = cluster0["TC3_3d"],
                    mode = "markers",
                    name = "Cluster 0",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text = None)

#trace2 es para 'Cluster 1'
trace2 = go.Scatter3d(
                    x = cluster1["TC1_3d"],
                    y = cluster1["TC2_3d"],
                    z = cluster1["TC3_3d"],
                    mode = "markers",
                    name = "Cluster 1",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text = None)

#trace3 es para 'Cluster 2'
trace3 = go.Scatter3d(
                    x = cluster2["TC1_3d"],
                    y = cluster2["TC2_3d"],
                    z = cluster2["TC3_3d"],
                    mode = "markers",
                    name = "Cluster 2",
                    marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
                    text = None)

#trace4 es para 'Cluster 3'
trace4 = go.Scatter3d(
                    x = cluster3["TC1_3d"],
                    y = cluster3["TC2_3d"],
                    z = cluster3["TC3_3d"],
                    mode = "markers",
                    name = "Cluster 3",
                    marker = dict(color = 'rgba(20, 80, 160, 0.8)'),
                    text = None)

data = [trace1, trace2, trace3, trace4]

title = "Visualisando Clusters en Tres Dimensiones Usando T-SNE (perplexity=" + str(perplexity) + ")"

layout = dict(title = title,
              xaxis= dict(title= 'TC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'TC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

### Comentarios sobre T-SNE:

El algoritmo T-SNE hizo un trabajo bastante decente en visualizar los clústeres también. Pero, había algunas diferencias notables al comparar sus gráficos resultantes con los gráficos resultantes de PCA.

Una gran diferencia entre los gráficos producidos por PCA y T-SNE es que los gráficos de T-SNE parecían tener sus clústeres superpuestos entre sí más que en los gráficos de PCA. Por ejemplo, si miras el gráfico 2-D formado por PCA, ves cuatro secciones distintas de los puntos de datos con fronteras estrictas y visibles que separan cada color en grupos. Mientras que, si miras el gráfico 2-D formado por T-SNE, vuelves a ver cuatro secciones formadas dentro de los puntos de datos, pero esta vez, los puntos de datos entre cada clúster parecen 'mezclarse' y superponerse más.

La otra gran diferencia entre los gráficos creados por PCA y los gráficos creados por T-SNE, es la forma. Debido a que tanto PCA como T-SNE realizan la reducción de dimensionalidad de maneras muy diferentes (y con objetivos diferentes), la forma o distribución resultante de los puntos producidos por los algoritmos casi siempre será muy diferente.

Ten en cuenta que los gráficos resultantes del algoritmo T-SNE son bastante variables, ya que dependen mucho del valor elegido para la `perplejidad`.

'1.5.2'